In [ ]:
import numpy as np
import pickle
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing, cross_validation
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import neighbors

import math

import matplotlib.pyplot as mt
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import warnings
warnings.filterwarnings("ignore")

In [ ]:

    train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])    
#     Removing outliers
#     train=train[train.Y!=90]

In [ ]:
 test = pd.read_csv('../input/test.csv', parse_dates=['Dates'])

In [ ]:
#     xy_scaler =preprocessing.StandardScaler()
#     xy_scaler.fit(train[["X","Y"]])
#     train[["X","Y"]]=xy_scaler.transform(train[["X","Y"]])
#     pca=PCA(2)
#     train.loc[:,["X","Y"]]=pca.fit_transform(pd.DataFrame(train.loc[:,["X","Y"]]))

In [ ]:
# dll=pd.DataFrame(list(set(train.Category)))
# dll
# train.head(5)
# labels.shape
# train.isna().count()

In [ ]:
# train.shape
# train.describe()

In [ ]:
train.Address.fillna('',inplace=True)
df = pd.Series([ x.replace('/','of') for x in train.Address.tolist() ])
df=pd.DataFrame([ x.split('of') for x in df.tolist()],columns=["Add1","Add2"])
train=pd.concat([train,df["Add1"],df["Add2"]],axis=1)
# train.head(5)

In [ ]:

vectorizer = HashingVectorizer(n_features=100)
# encode document
vector = vectorizer.transform(train.Address)
v1=vectorizer.transform(train.Add1)
v2=vectorizer.transform(train.Add2)

In [ ]:
svd=TruncatedSVD(n_components=2, random_state=42)
train["AddSVD"]=pd.DataFrame(svd.fit_transform(vector)).loc[:,1]
train["Add1SVD"]=pd.DataFrame(svd.fit_transform(v1)).loc[:,1]
train["Add2SVD"]=pd.DataFrame(svd.fit_transform(v2)).loc[:,1]

In [ ]:
train.Descript.fillna('',inplace=True)
vector = vectorizer.transform(train.Descript)
svd=TruncatedSVD(n_components=2, random_state=42)
train["DesSVD"]=pd.DataFrame(svd.fit_transform(vector)).loc[:,1]

In [ ]:
#     Removing outliers
# train=train[train.Y!=90]

In [ ]:
kmeans = KMeans(n_clusters=36,max_iter=300)
df=pd.DataFrame(pd.concat([pd.DataFrame(train.X),pd.DataFrame(train.Y)],axis=1))
kmeans.fit(df)
labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_


In [ ]:
# fig = mt.figure(figsize=(5, 5))

# # colors = map(lambda x: colmap[x+1], labels)

# mt.scatter(df['X'], df['Y'], alpha=0.5,cmap=mt.get_cmap('jet'), edgecolor='k')
# for idx, centroid in enumerate(centroids):
# #     mt.scatter(*centroid, color=colmap[idx+1])
#     mt.scatter(*centroid)
# # mt.xlim(0, 80)
# # mt.ylim(0, 80)
# mt.show()

In [ ]:
le_crime = preprocessing.LabelEncoder()
crime = le_crime.fit_transform(train.Category)

In [ ]:
# def divmon(m):
#     if (m>=1 and m<=2): return 'A'
#     if (m>=3 and m<=5): return 'B'
#     if (m>=5 and m<=9): return 'C'
#     if (m==10): return 'D'
#     if (m>=11 and m<=12): return 'E'
mon = train.Dates.dt.month
# mon = pd.DataFrame([divmon(m) for m in mon.tolist()])
mon=pd.get_dummies(mon,prefix='m')


In [ ]:

# crime = train.Category
crimelist = pd.get_dummies(train.Category) 
days = pd.get_dummies(train.DayOfWeek)
district = pd.get_dummies(train.PdDistrict)
# hour = train.Dates.dt.hour
hour = pd.get_dummies(train.Dates.dt.hour,prefix='h')
minu = train.Dates.dt.minute
# Klabels=pd.get_dummies(labels,prefix='kl')
# train["hour"] = train["hour"]*60+minu
# address = pd.get_dummies(train.Address)
# desc = pd.get_dummies(train.Descript)
minu = pd.get_dummies(minu)

train["Klabels"]=labels


In [ ]:
    train_data = pd.concat([minu,mon, train.Klabels,days,hour,district,train.X,train.Y,train.AddSVD,train.Add1SVD,train.Add2SVD,train.DesSVD], axis=1)
    train_data['crime'] = crime
    train_data = pd.concat([train_data,crimelist],axis=1)
#     train_data = pd.concat([train_data,Klabels],axis=1)
#     train_data['hour'] = hour
#     train_data['mon'] = mon

#     train_data.head(5)

In [ ]:
# train_data.fillna(0,inplace=True)
train_data.shape

In [ ]:
# train_data1 = pd.concat([train_data,pd.DataFrame(Klabels)],axis=1)
# train_data1=train_data.join(Klabels)
# for i in range(len(Klabels.columns)):
#     train_data1 = train_data.assign(Klabels.loc[:,i])
# train_data1.shape

In [ ]:
# train_data

In [ ]:
    days = pd.get_dummies(test.DayOfWeek)
    district = pd.get_dummies(test.PdDistrict)
    minu = test.Dates.dt.minute
    minu = pd.get_dummies(minu)
    hour = pd.get_dummies(test.Dates.dt.hour,prefix='h')
#     hour = test.Dates.dt.hour
#     test["hour"] = test["hour"]*60+minu

#     address = le_crime.fit_transform(test.Address)
#     desc = le_crime.fit_transform(test.Descript)
    
    mon = test.Dates.dt.month
#     mon = pd.DataFrame([divmon(m) for m in mon.tolist()])

    mon=pd.get_dummies(mon,prefix='m')
    
#     test[["X","Y"]]=xy_scaler.fit_transform(test[["X","Y"]])
#     pca=PCA(2)
#     test.loc[:,["X","Y"]]=pca.fit_transform(pd.DataFrame(test.loc[:,["X","Y"]]))
    
    d = pd.Series([ x.replace('/','of') for x in test.Address.tolist() ])
    d=pd.DataFrame([ x.split('of') for x in d.tolist()],columns=["Add1","Add2"])
    test=pd.concat([test,d["Add1"],d["Add2"]],axis=1)
    
    vector = vectorizer.transform(test.Address)
    v1=vectorizer.transform(test.Add1)
    v2=vectorizer.transform(test.Add2)
    
    svd=TruncatedSVD(n_components=10, random_state=42)
    test["AddSVD"]=pd.DataFrame(svd.fit_transform(vector)).loc[:,1]
    test["Add1SVD"]=pd.DataFrame(svd.fit_transform(v1)).loc[:,1]
    test["Add2SVD"]=pd.DataFrame(svd.fit_transform(v2)).loc[:,1]
    
    vector = vectorizer.transform(test.Descript)
    test["DesSVD"]=pd.DataFrame(svd.fit_transform(vector)).loc[:,1]

    df=pd.DataFrame(pd.concat([pd.DataFrame(test.X),pd.DataFrame(test.Y)],axis=1))
#     kmeans = KMeans(n_clusters=200,max_iter=300)
    kmeans.fit(df)
    labels = kmeans.predict(df)
    centroids = kmeans.cluster_centers_
    test["Klabels"]=labels
#     Klabels=pd.get_dummies(labels,prefix='kl')

In [ ]:
    test_data = pd.concat([test.Id,test.Klabels,minu,mon,hour,days,district,test.X,test.Y,test.AddSVD,test.Add1SVD,test.Add2SVD,test.DesSVD], axis=1)
#     test_data = pd.concat([test_data,Klabels],axis=1)
#     test_data['hour'] = hour
# #     test_data['mon'] = mon
# #     test_data.columns

In [ ]:
features1 = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday','Wednesday', 
            'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION','NORTHERN', 'PARK', 'RICHMOND',
             'SOUTHERN', 'TARAVAL','TENDERLOIN'] + [x for x in range(0, 59)]+ [x for x in (set(pd.DataFrame(hour).columns))]
# features1 = features1 + [x for x in (set(pd.DataFrame(Klabels).columns))]
# +[x for x in (set(pd.DataFrame(mon).columns))]
#             ,'Add2SVD'
#     ,'hour'
features2 = ['AddSVD','Klabels','X','Y','DesSVD']
features = features1+features2
features3 = ['hour','minu','district','Y','DesSVD']

In [ ]:
# train_data

In [ ]:
# le_crime = preprocessing.LabelEncoder()
#     crime = le_crime.fit_transform(train.Category)
# crime = train.Category
# crimelist = pd.get_dummies(train.Category) 
lday=preprocessing.LabelEncoder()
ldistrict=preprocessing.LabelEncoder()
# days = pd.get_dummies(train.DayOfWeek)
train["days"] = lday.fit_transform(train.DayOfWeek)
# district = pd.get_dummies(train.PdDistrict)
train["district"] = ldistrict.fit_transform(train.PdDistrict)
train["hour"] = train.Dates.dt.hour
train["minu"] = train.Dates.dt.minute

train_data1 = pd.concat([crimelist,train.hour,train.minu,train.days,train.district,train.X,train.Y,train.DesSVD], axis=1)

test["days"] = lday.fit_transform(test.DayOfWeek)
# district = pd.get_dummies(train.PdDistrict)
test["district"] = ldistrict.transform(test.PdDistrict)
test["hour"] = test.Dates.dt.hour
test["minu"] = test.Dates.dt.minute

test_data1 = pd.concat([test.Id,test.hour,test.minu,test.days,test.district,test.X,test.Y,test.DesSVD], axis=1)
  

In [ ]:
count=0
ll= pd.DataFrame(np.random.normal(0,1,(5,6)),columns=["Training Size","Log LossBNB","Log LossGNB","StackedLR","KNN","Log LossMIX"])
# ll= pd.DataFrame(np.random.normal(0,1,(16,2)),columns=["Neighbours","Log Loss"])

train_data[features].head(5)

In [ ]:
count=0
train_data = shuffle(train_data)
sampletest,remtrain = train_test_split(train_data, test_size=(0.2))
for j in range(70,95,20):
    training, validation = train_test_split(remtrain, train_size=(j/100))
#     training = train_data
#     validation = test_data
#     train_data1 = shuffle(train_data1)
#     training1, validation1 = train_test_split(train_data1, train_size=(j/100))
    
    ll.at[count,["Training Size"]]=str(j)
    model = GaussianNB()
    model.fit(training[features2], training['crime'])
    predicted = np.array(model.predict_proba(validation[features2]))
    ll.at[count,["Log LossGNB"]]=str(log_loss(validation['crime'], predicted))
    modelB = BernoulliNB()
    modelB.fit(training[features], training['crime'])
    predictedB = np.array(modelB.predict_proba(validation[features]))
    ll.at[count,["Log LossBNB"]]=str(log_loss(validation['crime'], predictedB))
    
#     Logistic Regression:- Stacking with bernoulli 
    logreg = LogisticRegression()
    logreg.fit(modelB.predict(validation[features]).reshape(-1,1),validation['crime'])
#     modelB.fit(remtrain[features], remtrain['crime'])
#     predictedB = np.array(modelB.predict_proba(sampletest[features]))
    predictedSLR=logreg.predict_proba(modelB.predict(sampletest[features]).reshape(-1,1))
    ll.at[count,["StackedLR"]]=str(log_loss(sampletest['crime'], predictedSLR))
    
#     modelMNB = MultinomialNB()
    modelBNB = BernoulliNB()
    modelGNB = GaussianNB()
    modelBNB.fit(training[features1], training['crime'])
    modelGNB.fit(training[features2], training['crime'])
#     modelMNB.fit(training[features5], training['crime'])
#     calculating the log loss function with the validation section and printing the value
# class_prior Prior probabilities of the classes. If specified the priors are not adjusted according to the data.
    pos_prior = modelGNB.class_prior_
    predictedBNB = np.array(modelBNB.predict_proba(validation[features1]))
    predictedGNB = np.array(modelGNB.predict_proba(validation[features2]))
#     predictedMNB = np.array(modelMNB.predict_proba(validation[features5]))
#     predicted = (predictedBNB)* (predictedGNB/pos_prior)
    predicted = (predictedBNB)* (predictedGNB)
    ll.at[count,["Log LossMIX"]]=str(log_loss(validation['crime'], predicted))
    
   
#     model = MultinomialNB()
#     model.fit(training1[features3], training1['crime'])
#     predictedm = np.array(model.predict_proba(validation1[features3]))
#     ll.at[count,["Log LossMNB"]]=str(log_loss(validation1['crime'], predictedm))
    
    # KNN
    train_data1 = shuffle(train_data1)
    training1, validation1 = train_test_split(train_data1, train_size=.80)
    n_neighbors=1300
    # for n_neighbors in range(1000,2010,500):
    # for i, weights in enumerate(['uniform', 'distance']):
    knn = neighbors.KNeighborsRegressor(n_neighbors,n_jobs=500,weights='distance')
    knn.fit(training1[features3],training1[list(pd.DataFrame(crimelist).columns)])
    predicted = knn.predict(validation1[features3])
    ll.at[count,["KNN"]]=str(log_loss(validation1[list(pd.DataFrame(crimelist).columns)], predicted))
    
    
    count=count+1


In [ ]:
ll
# (pd.DataFrame(sampletest['crime'])).values
# (pd.DataFrame(modelB.predict(validation[features])).values)


In [ ]:
# # save the model to disk
# filename = 'BernoulliNB_all_fN.sav'
# pickle.dump(modelB, open(filename, 'wb'))
# filename = 'BNBN.sav'
# pickle.dump(modelBNB, open(filename, 'wb'))
# filename = 'GNBN.sav'
# pickle.dump(modelGNB, open(filename, 'wb'))
# filename = 'KNN.sav'
# pickle.dump(modelB, open(filename, 'wb'))

In [ ]:
# # KNN
# # train_data = shuffle(train_data)
# # training, validation = train_test_split(train_data, train_size=.80)
# n_neighbors=1500
# # for n_neighbors in range(2000,2010,1000):
# # for i, weights in enumerate(['uniform', 'distance']):
# knn = neighbors.KNeighborsRegressor(n_neighbors,n_jobs=500)
# knn.fit(training[features],training['crime'])
# predicted = knn.predict(validation[features])
# ll.at[count,["Neighbours"]]=str(n_neighbors)
# ll.at[count,["Log Loss"]]=str(log_loss(validation[list(pd.DataFrame(crime).columns)], predicted))
# count=count+1

In [ ]:
# crime = le_crime.transform(train.Category)
# pd.DataFrame(predicted).head(5)


In [ ]:
# ll

In [ ]:
# model = BernoulliNB()
# model.fit(train_data[features], train_data['crime'])
# predicted = model.predict_proba(test_data[features])
# # predicted=pd.DataFrame(predicted, columns = le_crime.classes_)
# # result=pd.concat([predicted,test_data['Id']],axis=1)
# # # result.to_csv('~/Desktop/San Files/submitNB.csv')
# # modelBNB = BernoulliNB()
# # modelGNB = GaussianNB()
# # modelBNB.fit(train_data[features1], train_data['crime'])
# # modelGNB.fit(train_data[features2], train_data['crime'])
# # pos_prior = modelGNB.class_prior_
# # predictedBNB = np.array(modelBNB.predict_proba(test_data[features1]))
# # predictedGNB = np.array(modelGNB.predict_proba(test_data[features2]))
# # predicted = (predictedBNB)* ( predictedGNB/pos_prior)
# predicted = knn.predict(test_data1[features3])
# predicted = pd.DataFrame(predicted, columns = list(pd.DataFrame(crimelist).columns))
# # predicted=pd.DataFrame(predicted, columns = le_crime.classes_)
# # pd.DataFrame(predicted).head(5)
# result=pd.concat([predicted,test_data1['Id']],axis=1)
# result.to_csv('submitKNN.csv')

In [ ]:
# KNN 
# n_neighbors=10
# knn = neighbors.KNeighborsRegressor(n_neighbors)
# knn.fit(train_data[features],train_data[list(pd.DataFrame(crime).columns)])
# predicted = knn.predict(test_data[features])
# predicted = pd.DataFrame(predicted, columns = list(pd.DataFrame(crime).columns))
# result=pd.concat([predicted,test_data['Id']],axis=1)
# result.to_csv('~/Desktop/San Files/submitNB.csv')

In [ ]:
# pd.DataFrame(predicted)

In [ ]:
#     predicted_val = model.predict(test_data[features])
#     predicted_val = le_crime.inverse_transform(predicted_val)
#     result = pd.DataFrame(predicted_val)
#     result.to_csv('~/Desktop/San Files/resultsNB.csv', index=False)
    